## Analyze Morbidmap content - v3
The goal of this notebook is to analyze the content of the files from OMIM called morbidmap and mimTitles in order to create a gold standard list of diseases that should be represented in Mondo with 'has material basis in germline mutation in' some GENE. The diseases in this list can be used for comparison of results that occur through the various transformations of the omim content to confirm the final representation is correct in downstream files, e.g. omim.owl.

To download these input files (morbidmap and mimTitles), request an API key from OMIM (https://omim.org/contact#) and then create the files using python -m omim2obo based on the instructions in the README in the omim repo.

For this analysis, the working assumption from Sabrina's latest email ('Gene association in Mondo' on Fri, Nov 1, 6:46 PM) is that the gene associations to add into Mondo are:
1) The disease has exactly 1 associated gene
2) The association is causal (mapping key = 3)
3) Classified as a disease, non-provisional, and not a susceptibility relationshsip (phenotype label does NOT include [], {}, or ?)
 
See https://omim.org/help/faq#1_6 for more details on what the Phenotype mapping key values mean and additional formatting, [], {}, ?, found in phenotype labels. See https://omim.org/help/faq#1_3 for information on what the Prefix values in the file mimTitles means.

NOTE: Without filtering out from this set diseases where the phenotype label contains 'digenic' there will be a handful of these since
as we saw in Analyze Morbidmap content - v1 these exist. Also in the summary doc of [OMIM Disease-Gene Issues](https://docs.google.com/document/d/1cLfBgPIZWiN5LX-E-xwSyBeFdT-vw0JuSfSM7HL3_hc/edit?tab=t.0#heading=h.h4y343h64cck).

Also, without filtering out from this set diseases that start with [,{, and ? non-diseases, susceptibility, and provisional diseases 
will be included. See https://omim.org/help/faq#1_6 for a description of these special characters.

### Imports

In [1]:
# Imports
import pandas as pd
import re
import time

# Set the display option to show full column width
pd.set_option('display.max_colwidth', None)

### Read in Data file

In [2]:
# Read in file. This version of morbidmap.tsv was downloaded on 18-Nov-2024
# NOTE: You will need to follow the instructions in the README to get the morbidmap file. 
# IMPORTANT !!The morbidmap file is not a file that should be posted publicly in this repo!!

df = pd.read_csv('../../data/morbidmap.tsv', sep='\t')
df.head()

,Phenotype,Gene/Locus And Other Related Symbols,MIM Number,Cyto Location
0,"17,20-lyase deficiency, isolated, 202110 (3)","CYP17A1, CYP17, P450C17",609300,10q24.32
1,"17-alpha-hydroxylase/17,20-lyase deficiency, 202110 (3)","CYP17A1, CYP17, P450C17",609300,10q24.32
2,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787,5p13.2
3,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301,10q26.13
4,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577,6p21.1


### Process file to parse out phenotype mim number from Phenotype column

In [3]:
# Parse out phenotype mim number from Phenotype column

# Updated pattern - from https://github.com/monarch-initiative/omim/pull/158/files#diff-712ded77b46725c43257568450e7c94df2a64f683c77dfd88e9726fbcbc7c5fbR351
pattern = r'(.*)(\d{6})\s*(?:\((\d+)\))?'

# Use .str.extract() to apply the pattern and store matches in new columns
df[['p_label', 'p_mim', 'p_mapping_key']] = df['Phenotype'].str.extract(pattern)

df.head()

,Phenotype,Gene/Locus And Other Related Symbols,MIM Number,Cyto Location,p_label,p_mim,p_mapping_key
0,"17,20-lyase deficiency, isolated, 202110 (3)","CYP17A1, CYP17, P450C17",609300,10q24.32,"17,20-lyase deficiency, isolated,",202110,3
1,"17-alpha-hydroxylase/17,20-lyase deficiency, 202110 (3)","CYP17A1, CYP17, P450C17",609300,10q24.32,"17-alpha-hydroxylase/17,20-lyase deficiency,",202110,3
2,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787,5p13.2,"2,4-dienoyl-CoA reductase deficiency,",616034,3
3,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301,10q26.13,"2-methylbutyrylglycinuria,",610006,3
4,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577,6p21.1,"3-M syndrome 1,",273750,3


In [4]:
# Convert type of p_mapping_key to a string

df['p_mapping_key'] = df['p_mapping_key'].astype(str)

# Check that each value is now a string
print(df['p_mapping_key'].apply(type).unique())

[<class 'str'>]


### Get all rows where the p_mim value occurs only 1 time in the dataframe and has p_mapping_key='3'

In [5]:
# Step 1: Filter for rows where p_mim occurs only once and p_mapping_key is 3
unique_p_mim = df['p_mim'].value_counts()[df['p_mim'].value_counts() == 1].index
print("Length of unique_p_mim: ", len(unique_p_mim))

unique_and_pkey3_df = df[(df['p_mim'].isin(unique_p_mim)) & (df['p_mapping_key'] == '3')]
print ("\n---\nunique_and_pkey3_df.nunique() values:")
print(unique_and_pkey3_df.nunique())

unique_and_pkey3_df.head()

Length of unique_p_mim:  6350

---
unique_and_pkey3_df.nunique() values:
Phenotype                               6331
Gene/Locus And Other Related Symbols    4622
MIM Number                              4622
Cyto Location                            834
p_label                                 6330
p_mim                                   6331
p_mapping_key                              1
dtype: int64


,Phenotype,Gene/Locus And Other Related Symbols,MIM Number,Cyto Location,p_label,p_mim,p_mapping_key
2,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787,5p13.2,"2,4-dienoyl-CoA reductase deficiency,",616034,3
3,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301,10q26.13,"2-methylbutyrylglycinuria,",610006,3
4,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577,6p21.1,"3-M syndrome 1,",273750,3
5,"3-M syndrome 2, 612921 (3)","OBSL1, KIAA0657, 3M2",610991,2q35,"3-M syndrome 2,",612921,3
6,"3-M syndrome 3, 614205 (3)","CCDC8, 3M3",614145,19q13.32,"3-M syndrome 3,",614205,3


In [6]:
# Remove rows where the p_label starts with an "OMIM special character". See https://omim.org/help/faq#1_6

# Filter out rows where p_label starts with [, {, or ?
unique_and_key3_filtered_df = unique_and_pkey3_df[~unique_and_pkey3_df['p_label'].str.match(r'^[\[{?]', na=False)]
print(unique_and_key3_filtered_df.nunique())

unique_and_key3_filtered_df.head()

Phenotype                               5340
Gene/Locus And Other Related Symbols    4014
MIM Number                              4014
Cyto Location                            811
p_label                                 5339
p_mim                                   5340
p_mapping_key                              1
dtype: int64


,Phenotype,Gene/Locus And Other Related Symbols,MIM Number,Cyto Location,p_label,p_mim,p_mapping_key
2,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787,5p13.2,"2,4-dienoyl-CoA reductase deficiency,",616034,3
3,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301,10q26.13,"2-methylbutyrylglycinuria,",610006,3
4,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577,6p21.1,"3-M syndrome 1,",273750,3
5,"3-M syndrome 2, 612921 (3)","OBSL1, KIAA0657, 3M2",610991,2q35,"3-M syndrome 2,",612921,3
6,"3-M syndrome 3, 614205 (3)","CCDC8, 3M3",614145,19q13.32,"3-M syndrome 3,",614205,3


In [7]:
# Read in mimTitles to join with unique_and_key3_no_digenic_filtered_df to get Prefix values
mimTitles_df = pd.read_csv('../../data/mimTitles.tsv', sep='\t')
print(mimTitles_df.nunique())

mimTitles_df.head()

Prefix                                 5
MIM Number                         28952
Preferred Title; symbol            28684
Alternative Title(s); symbol(s)    18991
Included Title(s); symbols          1314
dtype: int64


,Prefix,MIM Number,Preferred Title; symbol,Alternative Title(s); symbol(s),Included Title(s); symbols
0,NaN,100050,"AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,NaN
1,Percent,100070,"AORTIC ANEURYSM, FAMILIAL ABDOMINAL, 1; AAA1","ANEURYSM, ABDOMINAL AORTIC; AAA;; ABDOMINAL AORTIC ANEURYSM",NaN
2,Number Sign,100100,PRUNE BELLY SYNDROME; PBS,"ABDOMINAL MUSCLES, ABSENCE OF, WITH URINARY TRACT ABNORMALITY AND CRYPTORCHIDISM;; EAGLE-BARRETT SYNDROME; EGBRS",NaN
3,NaN,100200,ABDUCENS PALSY,NaN,NaN
4,Number Sign,100300,ADAMS-OLIVER SYNDROME 1; AOS1,"AOS;; ABSENCE DEFECT OF LIMBS, SCALP, AND SKULL;; CONGENITAL SCALP DEFECTS WITH DISTAL LIMB REDUCTION ANOMALIES;; APLASIA CUTIS CONGENITA WITH TERMINAL TRANSVERSE LIMB DEFECTS","APLASIA CUTIS CONGENITA, CONGENITAL HEART DEFECT, AND FRONTONASAL CYSTS, INCLUDED"


In [8]:
# Merge dataframes

# Ensure both p_mim and MIM Number columns are of the same data type (string in this example)
unique_and_key3_filtered_df['p_mim'] = unique_and_key3_filtered_df['p_mim'].astype(str)
mimTitles_df['MIM Number'] = mimTitles_df['MIM Number'].astype(str)

# Perform the join based on p_mim and MIM Number
merged_df = unique_and_key3_filtered_df.merge(
    mimTitles_df, left_on='p_mim', right_on='MIM Number', how='left'
)

print(merged_df.nunique())

merged_df.head()

Phenotype                               5340
Gene/Locus And Other Related Symbols    4014
MIM Number_x                            4014
Cyto Location                            811
p_label                                 5339
p_mim                                   5340
p_mapping_key                              1
Prefix                                     2
MIM Number_y                            5340
Preferred Title; symbol                 5340
Alternative Title(s); symbol(s)         2900
Included Title(s); symbols               219
dtype: int64


/var/folders/cp/m4__ys497773m0zyz5l__yqw0000gq/T/ipykernel_76076/112016101.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_and_key3_filtered_df['p_mim'] = unique_and_key3_filtered_df['p_mim'].astype(str)


,Phenotype,Gene/Locus And Other Related Symbols,MIM Number_x,Cyto Location,p_label,p_mim,p_mapping_key,Prefix,MIM Number_y,Preferred Title; symbol,Alternative Title(s); symbol(s),Included Title(s); symbols
0,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787,5p13.2,"2,4-dienoyl-CoA reductase deficiency,",616034,3,Number Sign,616034,"2,4-DIENOYL-CoA REDUCTASE DEFICIENCY; DECRD",NaN,NaN
1,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301,10q26.13,"2-methylbutyrylglycinuria,",610006,3,Number Sign,610006,2-METHYLBUTYRYL-CoA DEHYDROGENASE DEFICIENCY,2-METHYLBUTYRYL GLYCINURIA;; SHORT/BRANCHED-CHAIN ACYL-CoA DEHYDROGENASE DEFICIENCY; SBCADD,NaN
2,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577,6p21.1,"3-M syndrome 1,",273750,3,Number Sign,273750,THREE M SYNDROME 1; 3M1,3M SYNDROME;; LE MERRER SYNDROME;; DOLICHOSPONDYLIC DYSPLASIA;; GLOOMY FACE SYNDROME,"YAKUT SHORT STATURE SYNDROME, INCLUDED"
3,"3-M syndrome 2, 612921 (3)","OBSL1, KIAA0657, 3M2",610991,2q35,"3-M syndrome 2,",612921,3,Number Sign,612921,THREE M SYNDROME 2; 3M2,3M SYNDROME 2,NaN
4,"3-M syndrome 3, 614205 (3)","CCDC8, 3M3",614145,19q13.32,"3-M syndrome 3,",614205,3,Number Sign,614205,THREE M SYNDROME 3; 3M3,3M SYNDROME 3,NaN


In [9]:
# Modify to keep only certain columns

# Specify the columns you want to keep
columns_to_keep = ['Phenotype', 'Gene/Locus And Other Related Symbols', 'MIM Number_x',  'Cyto Location', 'p_label',
                   'p_mim', 'p_mapping_key', 'Prefix', 'MIM Number_y']

# Create a new DataFrame with only these columns
new_df = merged_df[columns_to_keep]

# Re-name columns
new_df = new_df.rename(columns={
    'MIM Number_x': 'Gene MIM',
    'MIM Number_y': 'Phenotype MIM'
})

print(new_df.nunique())

new_df.head()

Phenotype                               5340
Gene/Locus And Other Related Symbols    4014
Gene MIM                                4014
Cyto Location                            811
p_label                                 5339
p_mim                                   5340
p_mapping_key                              1
Prefix                                     2
Phenotype MIM                           5340
dtype: int64


,Phenotype,Gene/Locus And Other Related Symbols,Gene MIM,Cyto Location,p_label,p_mim,p_mapping_key,Prefix,Phenotype MIM
0,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787,5p13.2,"2,4-dienoyl-CoA reductase deficiency,",616034,3,Number Sign,616034
1,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301,10q26.13,"2-methylbutyrylglycinuria,",610006,3,Number Sign,610006
2,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577,6p21.1,"3-M syndrome 1,",273750,3,Number Sign,273750
3,"3-M syndrome 2, 612921 (3)","OBSL1, KIAA0657, 3M2",610991,2q35,"3-M syndrome 2,",612921,3,Number Sign,612921
4,"3-M syndrome 3, 614205 (3)","CCDC8, 3M3",614145,19q13.32,"3-M syndrome 3,",614205,3,Number Sign,614205


In [10]:
# Prefix has two values, let's see what these are:

print(new_df['Prefix'].unique())

['Number Sign' 'Percent']


In [11]:
# Save to file
timestamp = time.time()

new_df.to_csv(f'unique_and_key3_filtered_df_{timestamp}.tsv', sep='\t', index=False)